In [13]:
# while ($true) {
#     nvidia-smi
#     Start-Sleep -Seconds 1
# }

In [14]:
import json
import pandas as pd

In [15]:
# Уже подготовленный датасет
dataset = r'D:\Veeple\IT\Python_scripts\Scripts\Обучение aidar_musin\инфа из Согласия\try_2\mixed_dataset.json'

In [16]:
with open(dataset, "r", encoding="utf-8") as f:
    mixed_dataset = json.load(f)

In [17]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

# Загрузка модели и токенизатора
model_name = "aidarmusin/address-ner-ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Текущие метки модели
current_labels = model.config.id2label
print("Текущие метки модели:", current_labels)

# Добавление новых меток
new_labels = {
    0: "O",
    1: "B-ZipCode",
    2: "I-ZipCode",
    3: "B-Country",
    4: "I-Country",
    5: "B-Region",
    6: "I-Region",
    7: "B-District",
    8: "I-District",
    9: "B-Settlement",
    10: "I-Settlement",
    11: "B-Street",
    12: "I-Street",
    13: "B-House",
    14: "I-House",
    15: "B-Building",
    16: "I-Building",
    17: "B-Apartment",
    18: "I-Apartment",
    19: "B-Organization",  # Новая метка
    20: "I-Organization",  # Новая метка
}

# Обновление конфигурации
model.config.id2label = new_labels
model.config.label2id = {v: k for k, v in new_labels.items()}

# Увеличиваем количество классов в модели
model.num_labels = len(new_labels)
model.classifier = torch.nn.Linear(model.config.hidden_size, len(new_labels))

Текущие метки модели: {0: 'O', 1: 'B-ZipCode', 2: 'I-ZipCode', 3: 'B-Country', 4: 'I-Country', 5: 'B-Region', 6: 'I-Region', 7: 'B-District', 8: 'I-District', 9: 'B-Settlement', 10: 'I-Settlement', 11: 'B-Street', 12: 'I-Street', 13: 'B-House', 14: 'I-House', 15: 'B-Building', 16: 'I-Building', 17: 'B-Apartment', 18: 'I-Apartment'}


In [18]:
def prepare_data(dataset, tokenizer, model, max_length=64):
    inputs = []
    labels = []
    
    for item in dataset:
        address = item["address"]
        entities = item["extracted_entities"]
        
        # Токенизация с разбиением на токены и их смещениями
        encoding = tokenizer(
            address,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_offsets_mapping=True  # Важно! Нам нужны индексы токенов
        )
        
        offset_mapping = encoding["offset_mapping"].squeeze(0)  # Получаем смещения токенов
        label_ids = [0] * max_length  # Все токены по умолчанию "O"
        
        for entity in entities:
            ent_start, ent_end = entity["start"], entity["end"]
            ent_type = entity["type"]
            
            # Получаем ID для сущности (если нет в словаре, используем "O")
            label_id_B = model.config.label2id.get(f"B-{ent_type}", 0)
            label_id_I = model.config.label2id.get(f"I-{ent_type}", 0)
            
            # Назначаем метки для соответствующих токенов
            for i, (tok_start, tok_end) in enumerate(offset_mapping):
                if tok_start == 0 and tok_end == 0:
                    continue  # Пропускаем паддинги
                if tok_start >= ent_start and tok_end <= ent_end:
                    if label_ids[i] == 0:  # Если еще не размечен
                        label_ids[i] = label_id_B  # Первый токен как "B-"
                    else:
                        label_ids[i] = label_id_I  # Остальные "I-"
        
        labels.append(torch.tensor(label_ids))
        inputs.append(encoding)
    
    return inputs, labels


In [19]:
# Подготовка данных
max_length = 64  # Максимальная длина последовательности
inputs, labels = prepare_data(mixed_dataset, tokenizer, model, max_length=64)


In [20]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs[idx]["input_ids"].squeeze(),  # Убираем лишнюю размерность
            "attention_mask": self.inputs[idx]["attention_mask"].squeeze(),
            "labels": self.labels[idx],
        }

# Создание датасета
train_dataset = CustomDataset(inputs, labels)

In [21]:
from sklearn.model_selection import train_test_split

# Разделение данных на обучающую и оценочную части
train_data, eval_data = train_test_split(mixed_dataset, test_size=0.2, random_state=42)

# Подготовка обучающего набора данных
train_inputs, train_labels = prepare_data(train_data, tokenizer, model, max_length=64)
train_dataset = CustomDataset(train_inputs, train_labels)

# Подготовка оценочного набора данных
eval_inputs, eval_labels = prepare_data(eval_data, tokenizer, model, max_length=64)
eval_dataset = CustomDataset(eval_inputs, eval_labels)

In [22]:
from transformers import Trainer, TrainingArguments

# Определение TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",            # Каталог для сохранения модели
    evaluation_strategy="epoch",       # Оценка после каждой эпохи
    save_strategy="epoch",             # Сохранение модели после каждой эпохи
    learning_rate=5e-5,                # Можно немного уменьшить, т.к. дообучаем
    per_device_train_batch_size=16,    # Можно уменьшить до 8, если памяти не хватает
    per_device_eval_batch_size=16,     # Аналогично для оценки
    num_train_epochs=5,                # 5 эпох — неплохой баланс
    weight_decay=0.01,                 # Регуляризация для предотвращения переобучения
    save_total_limit=2,                 # Храним только 2 последние версии модели
    logging_dir="./logs",              # Каталог для логов
    logging_steps=10,                   # Логирование каждые 10 шагов
    warmup_steps=500,                   # Количество шагов для разогрева learning rate
    fp16=True,                          # Используем 16-битные тензоры (ускоряет обучение на GPU)
    gradient_accumulation_steps=2,      # Сглаживание градиентов, если маленький batch_size
)


# Создание Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)


In [ ]:
# Обучение модели
trainer.train()

In [ ]:
# Сохраняем модель и токенизатор в указанную папку
model.save_pretrained("./trained_try_2")
tokenizer.save_pretrained("./trained_try_2")

In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# Загружаем свою дообученную модель и токенизатор
model = AutoModelForTokenClassification.from_pretrained("./trained_try_2")
tokenizer = AutoTokenizer.from_pretrained(model_name)  # Или свой токенизатор, если использовался другой

# Инициализация pipeline для NER с обученной моделью
address_ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, device=-1)  # device=0 для GPU, device=-1 для CPU



In [ ]:
# Адрес для предсказания
address = "ООО Ромашка на Тверской ул. Тверская, д. 1"

# Применяем pipeline для извлечения сущностей
entities = address_ner_pipeline(address)

# Выводим результат
print(entities)